In [1]:
# unit of diversion: cookie
# Invariant Metrics
# Number of Cookies (Ck); Number of Clicks (Cl); Click Through Prob (Cl/Ck)

In [2]:
# Evaluation Matrics
# Gross Conversation enrolled/Clicks; dmin=0.01
# Retention paid/enrolled; dmin=0.01
# Net Conversation paid/Clicks; dmin=0.0075

In [6]:
#Estimate the baseline

baseline = {"Cookies":40000,"Clicks":3200,"Enrollments":660,"CTP":0.08,"GConversion":0.20625,
           "Retention":0.53,"NConversion":0.109313}

r=5000/40000
baseline["Cookies"]=5000
baseline["Clicks"]=baseline["Clicks"]*r
baseline["Enrollments"]=baseline["Enrollments"]*r

print(baseline)

{'Cookies': 5000, 'Clicks': 400.0, 'Enrollments': 82.5, 'CTP': 0.08, 'GConversion': 0.20625, 'Retention': 0.53, 'NConversion': 0.109313}


In [18]:
import math
from scipy.stats import norm

In [17]:
sd_gconversation=round(math.sqrt(baseline["GConversion"]*(1-baseline["GConversion"])/400),4)
print(sd_gconversation)

sd_retention=round(math.sqrt(baseline["Retention"]*(1-baseline["Retention"])/82.4),4)
print(sd_retention)

sd_nconversation=round(math.sqrt(baseline["NConversion"]*(1-baseline["NConversion"])/400),4)
print(sd_nconversation)

0.0202
0.055
0.0156


In [38]:
print(round(norm.ppf(0.975),4))
print(round(norm.cdf(1.96,0,1),4))

1.96
0.975


In [42]:
#alpha=0.05 beta=0.2
def calculate_min_sample(alpha,beta,dmin,p):
    z_alpha=norm.ppf(1-alpha/2)
    z_beta=norm.ppf(1-beta)
    sd1=math.sqrt(2*p*(1-p))
    sd2=math.sqrt(p*(1-p)+(p+dmin)*(1-(p+dmin)))
    
    n=(z_alpha*sd1+z_beta*sd2)**2/dmin**2
    return n

In [52]:
n_gconversation=round(calculate_min_sample(0.05,0.2,0.01,0.20625),0)
n_retention=round(calculate_min_sample(0.05,0.2,0.01,0.53),0)
n_nconversation=round(calculate_min_sample(0.05,0.2,0.0075,0.109313),0)

print(n_gconversation) #need 25835 clicks
print(n_retention)     #need 39087 enrolls
print(n_nconversation)

25835.0
39087.0
27413.0


In [60]:
sample_size_gconversation=n_gconversation/(400/5000)*2 #clicks -> cookie
sample_size_retention=n_retention/0.20625/0.08*2 #enrolled -> clicks (Gross Conversation) -> cookie
sample_size_nconversation=n_nconversation/(400/5000)*2


print(round(sample_size_gconversation,0))
print(round(sample_size_retention,0)) #4M too large drop the metric
print(round(sample_size_nconversation,0))

645875.0
4737818.0
685325.0


In [63]:
#analyze data
import pandas as pd
control_df=pd.read_excel("/users/xinjianlu/downloads/final_project_results.xlsx",sheet_name=0)
exp_df=pd.read_excel("/users/xinjianlu/downloads/final_project_results.xlsx",sheet_name=1)

In [65]:
control_df.head(3)

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0


In [75]:
#sanity check: number of page view
# H0: p=0.5
pageview_control=control_df["Pageviews"].sum()
pageview_exp=exp_df["Pageviews"].sum()
pageview_sum=pageview_exp+pageview_control

p=0.5
alpha=0.05
p_hat=pageview_control/pageview_sum
print(round(p_hat,4))
sd=math.sqrt(p*(1-p)/pageview_sum)
me=norm.ppf(1-alpha/2)*sd
pv_lower,pv_upper=p-me,p+me
print(round(pv_lower,4),round(pv_upper,4)) #since 0.5 in (0.4988,0.5012) then can't reject null hypothesis

0.5006
0.4988 0.5012


In [76]:
# sanity check: number of clicks
# H0: p=0.5
clicks_control=control_df["Clicks"].sum()
clicks_exp=exp_df["Clicks"].sum()
clicks_sum=clicks_control+clicks_exp

p=0.5
alpha=0.05
p_hat=clicks_control/clicks_sum
print(round(p_hat,4))
sd=math.sqrt(p*(1-p)/clicks_sum)
me=norm.ppf(1-alpha/2)*sd
pv_lower,pv_upper=p-me,p+me
print(round(pv_lower,4),round(pv_upper,4)) #since 0.5 in (0.4988,0.5012) then can't reject null hypothesis

0.5005
0.4959 0.5041


In [86]:
# sanity check: CTP (click through prob)
# H0: d_exp-d_control=0
# pooled probability=(pi_1*n1+pi_2*n2/(n1+n2))
# pooled variance=sqrt(p_pooled*(1-p_pooled)*(1/n1+1/n2))
ctp_control=clicks_control/pageview_control
ctp_exp=clicks_exp/pageview_exp
p_pool=clicks_sum/pageview_sum

d_hat=ctp_exp-ctp_control
print(round(d_hat,4))
sd=math.sqrt(p_pool*(1-p_pool)*(1/pageview_exp+1/pageview_control))
me=norm.ppf(1-alpha/2)*sd
pv_lower,pv_upper=-me,+me
print(round(pv_lower,4),round(pv_upper,4)) #since 0.0001 in (-0.0013,0.0013) then can't reject null hypothesis

0.0001
-0.0013 0.0013


In [87]:
control_df.isnull().sum()

Date            0
Pageviews       0
Clicks          0
Enrollments    14
Payments       14
dtype: int64

In [93]:
exp_df.isnull().sum()

Date            0
Pageviews       0
Clicks          0
Enrollments    14
Payments       14
dtype: int64

In [92]:
#examing effect: Gross Conversation
#H0: d=0
clicks_control=control_df[control_df["Enrollments"].notna()]["Clicks"].sum()
clicks_exp=exp_df[exp_df["Enrollments"].notna()]["Clicks"].sum()

enrollments_control=control_df["Enrollments"].sum()
enrollments_exp=exp_df["Enrollments"].sum()

gc_control=enrollments_control/clicks_control
gc_exp=enrollments_exp/clicks_exp

gc_p_pooled=(enrollments_exp+enrollments_control)/(clicks_control+clicks_exp)
gc_sd_pooled=math.sqrt(gc_p_pooled*(1-gc_p_pooled)*(1/clicks_control+1/clicks_exp))
gc_lower,gc_upper=(gc_exp-gc_control)-1.96*gc_sd_pooled,(gc_exp-gc_control)+1.96*gc_sd_pooled

print(round(gc_lower,4),round(gc_upper,4)) #didn't include 0 thus it is statistical significant but 
# experiment group worse than control group

-0.0291 -0.012


In [96]:
#examing effect: Net Conversation
#H0: d=0
payments_control=control_df["Payments"].sum()
payments_exp=exp_df["Payments"].sum()

nc_control=payments_control/clicks_control
nc_exp=payments_exp/clicks_exp

nc_p_pooled=(payments_control+payments_exp)/(clicks_control+clicks_exp)
nc_sd_pooled=math.sqrt(nc_p_pooled*(1-nc_p_pooled)*(1/clicks_control+1/clicks_exp))
nc_lower,nc_upper=(nc_exp-nc_control)-1.96*nc_sd_pooled,(nc_exp-nc_control)+1.96*nc_sd_pooled

print(round(nc_lower,4),round(nc_upper,4))

# include 0 thus not statistically significant and upper bound less than d_min (not practically significant)

-0.0116 0.0019


In [109]:
## Sign Tests
full_df=control_df.merge(exp_df,on="Date",suffixes=("_control","_exp"))
print(len(full_df))
full_df=full_df[full_df["Enrollments_control"].notna()]
print(len(full_df))

37
23


In [113]:
full_df["GC"]=full_df.apply(lambda x: 1 if x["Enrollments_exp"]/x["Clicks_exp"]> \
            x["Enrollments_control"]/x["Clicks_control"] else 0,axis=1)

full_df["NC"]=full_df.apply(lambda x: 1 if x["Payments_exp"]/x["Clicks_exp"]> \
            x["Payments_control"]/x["Clicks_control"] else 0,axis=1)

In [114]:
full_df.head()

,Date,Pageviews_control,Clicks_control,Enrollments_control,Payments_control,Pageviews_exp,Clicks_exp,Enrollments_exp,Payments_exp,GC,NC
0,"Sat, Oct 11",7723,687,134.0,70.0,7716,686,105.0,34.0,0,0
1,"Sun, Oct 12",9102,779,147.0,70.0,9288,785,116.0,91.0,0,1
2,"Mon, Oct 13",10511,909,167.0,95.0,10480,884,145.0,79.0,0,0
3,"Tue, Oct 14",9871,836,156.0,105.0,9867,827,138.0,92.0,0,0
4,"Wed, Oct 15",10014,837,163.0,64.0,9793,832,140.0,94.0,0,1


In [137]:
print(len(full_df[full_df["GC"]==1]))
print(len(full_df[full_df["NC"]==1]))

4
10


In [154]:
def binomial_prob(n,x):
    p=math.factorial(n)/(math.factorial(x)*math.factorial(n-x))*0.5**x*0.5**(n-x)
    return p

def get_result(n,x):
    prob=0
    for i in range(0,x+1):
        prob=prob+binomial_prob(n,i)
    return 2*prob

In [158]:
print(get_result(23,4)) #significant
print(get_result(23,10)) #not significant

0.002599477767944336
0.6776394844055176
